# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3250465686382987682
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357433344
locality {
  bus_id: 1
}
incarnation: 9844974581233052855
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    # https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/text.py#L199-L251
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sequences) for sequences in x])
    return pad_sequences(sequences=x, maxlen=length, dtype="int32", padding="post", value=0.0)

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.

Loss function: sparse categorical cross entropy: https://stats.stackexchange.com/questions/326065/cross-entropy-vs-sparse-cross-entropy-when-to-use-one-over-the-other


### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gap between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    # Time distributed layer: https://keras.io/api/layers/recurrent_layers/time_distributed/
    # https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/
    # https://medium.com/smileinnovation/how-to-work-with-time-distributed-data-in-a-neural-network-b8b39aa4ce00
    # applies the same Dense layer to every time step.
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    rnn = GRU(units=200, return_sequences=True)(input_seq)
    # As for the Dense layer size, we choose french_vocab_size because we want to activate the unit of the corresponding word in the output layer for translation. 
    logits = TimeDistributed(Dense(french_vocab_size))(rnn) 
    
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1)) #(batch_size, sequence_length, output_dim)

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
print(simple_rnn_model.summary())
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 200)           121200    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 344)           69144     
_________________________________________________________________
activation_2 (Activation)    (None, 21, 344)           0         
Total params: 190,344
Trainable params: 190,344
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 10s 93us/step - loss: 2.7057 - acc: 0.4663 - val_loss: nan - val_acc: 0.5415
Epoch 2/10
110288/110288 [==============================] - 10s 88us/step -

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [33]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    embedding_size = 128
    rnn_units = 64
    
    # Add layers to the model: input->embedding->rnn->denselayer->logits->softmax
    input_seq = Input(input_shape[1:])
    
    embedding = Embedding(input_dim = english_vocab_size,
                          output_dim = embedding_size,
                          input_length = input_shape[1:][0])(input_seq)
    rnn = GRU(units=rnn_units, return_sequences=True)(embedding)
    logits = TimeDistributed(Dense(units=french_vocab_size))(rnn)
    
    model = Model(input_seq, Activation('softmax')(logits))
    
    # Compile Model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

# TODO: Reshape the input, input shape to embedding layer:(batch_size, sequence_length)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# TODO: Train the neural network
embed_rnn_model = embed_model(
    input_shape=tmp_x.shape,
    output_sequence_length=max_french_sequence_length,
    english_vocab_size=english_vocab_size,
    french_vocab_size=french_vocab_size)
print(embed_rnn_model.summary())
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 21)                0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 21, 128)           25472     
_________________________________________________________________
gru_25 (GRU)                 (None, 21, 64)            37056     
_________________________________________________________________
time_distributed_24 (TimeDis (None, 21, 344)           22360     
_________________________________________________________________
activation_24 (Activation)   (None, 21, 344)           0         
Total params: 84,888
Trainable params: 84,888
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 86us/step - loss: 3

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [35]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    rnn_units = 200
    
    input_seq = Input(input_shape[1:])
    rnn = Bidirectional(GRU(units=rnn_units, return_sequences=True))(input_seq)
    logits = TimeDistributed(Dense(units=french_vocab_size))(rnn)
    
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)

# reshape input size
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# TODO: Train and Print prediction(s)
bidirectional_rnn_model = bd_model(
    input_shape=tmp_x.shape,
    output_sequence_length=max_french_sequence_length,
    english_vocab_size=english_vocab_size,
    french_vocab_size=french_vocab_size)
print(bidirectional_rnn_model.summary())
bidirectional_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(bidirectional_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 21, 400)           242400    
_________________________________________________________________
time_distributed_26 (TimeDis (None, 21, 344)           137944    
_________________________________________________________________
activation_26 (Activation)   (None, 21, 344)           0         
Total params: 380,344
Trainable params: 380,344
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 18s 159us/step - loss: 2.2710 - acc: 0.5374 - val_loss: nan - val_acc: 0.6090
Epoch 2/10
110288/110288 [==============================] - 16s 145us/step

In [15]:
preproc_french_sentences.shape

(137861, 21, 1)

In [16]:
tmp_x.shape

(137861, 21)

In [34]:
tmp_x[:1]

array([[17, 23,  1,  8, 67,  4, 39,  7,  3,  1, 55,  2, 44,  0,  0,  0,  0,
         0,  0,  0,  0]], dtype=int32)

In [ ]:
tmp_x[0]

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

"… RNN Encoder-Decoder, consists of two recurrent neural networks (RNN) that act as an encoder and a decoder pair. The encoder maps a variable-length source sequence to a fixed-length vector, and the decoder maps the vector representation back to a variable-length target sequence.

— Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation, 2014."

Differences between ```return sequences``` and ```return states``` for LSTMs in keras
https://machinelearningmastery.com/return-sequences-and-return-states-for-lstms-in-keras/:

- return sequences return the hidden state output for each input time step.
    
- return state returns the hidden state output and cell state for the last input time step.
    
- return sequences and return state can be used at the same time.

Create an encoder-decoder model in the cell below.

**Note on Keras**:

There are various ways to give input to decoder. Largely two ways are consider

Give the final context vector of encoder as the same input vector to all decoder cells
Give each encoder cells output to each decoder cells via 1:1 mapping. i.e. first encoders output goes to first decoder inputs.
Here we consider the first scenario where each decoder gets the same context vector as input.

Here is an example of usage of repeat vector

```
dim = 256
enc_rnn = LSTM(dim)
repeat_enc_representation = RepeatVector(output_sequence_length)
dec_rnn = LSTM(dim, return_sequences = True)
```
Now what is happening over here is that each encoder output is 256 dimensional ...Since we have return_sequences as False set in LSTM, enc_rnn has just one final context vector.

Now RepeatVector's argument output_sequence_length specifies the number of times to repeat the input.

Now as decoder takes a (num_time_steps, num_dimension) format, in this case this will be (output_sequence_length,dim).



In [40]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    from keras.layers import LSTM
    
    # Hyperparameters
    learning_rate = 1e-3
    lstm_units = 500
    
    # Encoder: https://keras.io/guides/working_with_rnns/
    encoder_input_seq = Input(shape=input_shape[1:])
    encoder_output, state_h, state_c = LSTM(units=lstm_units,
                                           return_sequences=False,
                                           return_state=True)(encoder_input_seq, initial_state=None)
            # or for GRU cells, encoder_output, state_t = GRU(...)
    encoder_state = [state_h, state_c]
    
    # Decoder
    # input RepeatVector: https://keras.io/api/layers/reshaping_layers/repeat_vector/
    decoder_input_seq = RepeatVector(output_sequence_length)(encoder_output)
    decoder_output = LSTM(units=lstm_units,
                       return_sequences=True,
                       return_state=False)(decoder_input_seq, initial_state=encoder_state)
    logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_output)
    
    # Model 
    model = Model(inputs=encoder_input_seq, 
                  outputs=Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    return model
    
tests.test_encdec_model(encdec_model)

# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
print(encdec_rnn_model.summary())

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=40, validation_split=0.2)

# Prediction
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           (None, 21, 1)        0                                            
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, 500), (None, 1004000     input_32[0][0]                   
__________________________________________________________________________________________________
repeat_vector_5 (RepeatVector)  (None, 21, 500)      0           lstm_8[0][0]                     
__________________________________________________________________________________________________
lstm_9 (LSTM)                   (None, 21, 500)      2002000     repeat_vector_5[0][0]            
                                                                 lstm_8[0][1]                     
          

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [35]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    from keras.layers import LSTM, concatenate
    
    # Hyperparameters
    learning_rate = 1e-3
    embedding_input_size = 200
    embedding_size = 256
    lstm_units = 300
    dropout = 0.2
    
    # Input --> Encoder --> Embedding --> bidirectional LSTM layers --> Encoder output --> Decoder input --> Decoder RNN --> Decoder output
    # https://keras.io/api/layers/recurrent_layers/bidirectional/
    
    # ENCODER
    encoder_input_seq = Input(shape=input_shape[1:])
    encoder_embedding_seq = Embedding(input_dim=embedding_input_size,
                                  output_dim=embedding_size,
                                  input_length = input_shape[1:][0])(encoder_input_seq)    
    # forward LSTM layer
    forward_encoder_output, forward_state_h, forward_state_c = LSTM(units=lstm_units,
                                               dropout=dropout,
                                               return_sequences=False,
                                               return_state=True,
                                               go_backwards=False)(encoder_embedding_seq)
    # backward LSTM layer
    backward_encoder_output, backward_state_h, backward_state_c = LSTM(units=lstm_units,
                                               dropout=dropout,
                                               return_sequences=False,
                                               return_state=True,
                                               go_backwards=True)(encoder_embedding_seq)
    # *Can also use Keras Bidirectional layer wrappers Bidirection(LSTM(...), mode='concatenate')
    # concatenate 2 LSTM's outputs and states
    state_h = concatenate([forward_state_h, backward_state_h])
    state_c = concatenate([forward_state_c, backward_state_c])
    encoder_output = concatenate([forward_encoder_output, backward_encoder_output])
    
    # DECODER
    decoder_input_seq = RepeatVector(output_sequence_length)(encoder_output)
    # decoder LSTM layer
    decoder_output = LSTM(
                      units=lstm_units * 2,
                      return_sequences=True,
                      return_state=False)(decoder_input_seq, initial_state=[state_h, state_c])
    # Dense layer to get logits
    logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_output)
    
    # Model
    model = Model(encoder_input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_model_final(model_final)

print('Final Model Loaded')

tmp_x = pad(preproc_english_sentences, max_french_sequence_length) # directly input into embedding layer
#tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

final_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
print(final_model.summary())
final_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

print(logits_to_text(final_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           (None, 21)           0                                            
__________________________________________________________________________________________________
embedding_29 (Embedding)        (None, 21, 256)      51200       input_29[0][0]                   
__________________________________________________________________________________________________
lstm_71 (LSTM)                  [(None, 300), (None, 668400      embedding_29[0][0]               
__________________________________________________________________________________________________
lstm_72 (LSTM)                  [(None, 300), (None, 668400      embedding_29[0][0]               
__________________________________________________________________________________________

## Prediction (IMPLEMENTATION)

In [36]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    
    model = model_final(input_shape=x.shape, 
                        output_sequence_length = y.shape[1], 
                        english_vocab_size = len(x_tk.word_index), 
                        french_vocab_size = len(y_tk.word_index))
    print(model.summary())
    model.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)

    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_30 (InputLayer)           (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_30 (Embedding)        (None, 15, 256)      51200       input_30[0][0]                   
__________________________________________________________________________________________________
lstm_74 (LSTM)                  [(None, 300), (None, 668400      embedding_30[0][0]               
__________________________________________________________________________________________________
lstm_75 (LSTM)                  [(None, 300), (None, 668400      embedding_30[0][0]               
__________________________________________________________________________________________________
concatenat

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [37]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 401087 bytes to machine_translation.html',
 '[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html',
 '[NbConvertApp] Writing 328614 bytes to machine_translation-zh.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?